In [ ]:
import os
import cv2
import numpy as np
import img_preprocess
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as compare_ssim


def preprocessing(image):
    # 베스트 이미지
    best_img = cv2.imread(
        "./product_images/true_ok/GSY827AN7A1356_AAO11960K_PKT10_CM1EQSUA0012_20220711210457_DirectLight_OK.jpg"
    )
    # 비교 이미지
    input_img = cv2.imread(image)
    

    best_carrier, best_sensor = img_preprocess.find_contours(best_img, show=False, sensor=True)
    input_carrier, input_sensor = img_preprocess.find_contours(input_img, show=False, sensor=True)
    
    if np.any(input_sensor) == None:
        return []
    else:
        input_sensor
    
    input_sensor = cv2.resize(input_sensor, dsize=(1676, 1258))

    tempDiff = cv2.subtract(best_sensor, input_sensor)

    best_gray = cv2.cvtColor(best_sensor, cv2.COLOR_BGR2GRAY)
    input_gray = cv2.cvtColor(input_sensor, cv2.COLOR_BGR2GRAY)

    (score, diff) = compare_ssim(best_gray, input_gray, full=True)
    diff = (diff * 255).astype("uint8")

    thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[0]

    # 차이점 빨간색으로 칠하기
    tempDiff[thresh == 255] = [0, 0, 255]

    return tempDiff, thresh


def get_hists(img, mask=None, ranges=[0, 255]):
    """show image's distribution

    Args:
        img (3D/2D Array): image
        mask (cv2.inrange, optional): image where you wanna get hist. Defaults to None.
        ranges (list, optional): _description_. Defaults to [0, 255].

    Returns:
        hists (list): list of each component's hist
    """
    colors = ["b", "g", "r"]
    img_planes = cv2.split(img)
    hists = []
    for (p, c) in zip(img_planes, colors):
        try:
            hist = cv2.calcHist(
                [p],
                [0],
                mask,
                [256],
                ranges,
            )
            hists.append([hist, c])

        except:
            pass

    return hists


def make_mask(per, n):
    """이미지에 마진margin을 n만큼 설정해서 출력

    Args:
        per (ndarray): 이미지
        n (int): 마진margin

    Returns:
        ndarray: 마진을 설정한 이미지
    """
    mask = np.zeros(per.shape[:2], np.uint8)
    mask[n : per.shape[0] - n, n : per.shape[1] - n] = 255
    return mask


def detect_result(hist, num_OK, num_NG):
    if hist >= 100:
        num_NG += 1

    else:
        num_OK += 1
    
    return num_OK, num_NG


def model_hj(folder):
    num_OK = 0
    num_NG = 0
    result = 0
    
    file_path = "./product_images/" + folder + "/"
    img_paths = os.listdir(file_path)
    for i in range(len(img_paths)):
        tempdiff = preprocessing(file_path + img_paths[i])
        if tempdiff == []:
            num_NG += 1
        else:
            mask = make_mask(tempdiff, 10)
            hists = get_hists(tempdiff, mask=mask)

            hist = np.sum(hists[2][0][6:])
            num_OK, num_NG = detect_result(hist, num_OK, num_NG)
            result+=hist
    return print(f"==== Result of Detection ====\n     총 수량 : {format(num_OK+num_NG, ',')} EA\n     양품 수량 : {format(num_OK, ',')} EA\n     불량품 수량 : {format(num_NG, ',')} EA\n     양품률 : {round(num_OK/(num_OK+num_NG)*100)} %\n     불량률 : {round(num_NG/(num_OK+num_NG)*100)} %\n     평균 Hist : {round(result/(num_OK+num_NG)*100)}")

In [ ]:
model_hj('true_ng2')

C:\Users\USER\AppData\Local\Temp\ipykernel_14472\1468423174.py:110: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if tempdiff == []:


==== Result of Detection ====
     총 수량 : 624 EA
     양품 수량 : 4 EA
     불량품 수량 : 620 EA
     양품률 : 1 %
     불량률 : 99 %
     평균 Hist : 15957188


In [ ]:
model_hj('overkill2')

C:\Users\USER\AppData\Local\Temp\ipykernel_14472\1468423174.py:110: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if tempdiff == []:


==== Result of Detection ====
     총 수량 : 500 EA
     양품 수량 : 373 EA
     불량품 수량 : 127 EA
     양품률 : 75 %
     불량률 : 25 %
     평균 Hist : 285522


In [ ]:
model_hj('true_ok2')

C:\Users\USER\AppData\Local\Temp\ipykernel_14472\1468423174.py:110: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if tempdiff == []:


==== Result of Detection ====
     총 수량 : 1,096 EA
     양품 수량 : 850 EA
     불량품 수량 : 246 EA
     양품률 : 78 %
     불량률 : 22 %
     평균 Hist : 596094
